In [1]:
import boto3
import urllib3
import os

# 1. Suppress the InsecureRequestWarning for cleaner output
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 2. Get credentials from environment variables (set by your bash script)
endpoint = "http://local-s3-service.ezdata-system.svc.cluster.local:30000"
bucket_name = "ingest-bucket-1"

# 3. Initialize the S3 client
s3 = boto3.client(
    's3',
    endpoint_url=endpoint,
    verify=False  # This is equivalent to --no-verify-ssl
)

try:
    # 4. List objects in the bucket
    print(f"--- Listing contents of {bucket_name} ---")
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:
        for obj in response['Contents']:
            print(f"File: {obj['Key']}  |  Size: {obj['Size']} bytes")
    else:
        print("Bucket is empty or does not exist.")

except Exception as e:
    print(f"Error connecting to S3: {e}")

--- Listing contents of ingest-bucket-1 ---
File: 102_s3_token_bash.sh  |  Size: 2044 bytes


In [2]:
file_key = "102_s3_token_bash.sh"

try:
    # 4. Fetch the object
    print(f"--- Fetching content of: {file_key} ---\n")
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    
    # 5. Read and decode the streaming body
    file_content = response['Body'].read().decode('utf-8')
    
    print(file_content)
    print("\n--- End of File ---")

except Exception as e:
    print(f"Error reading file from S3: {e}")

--- Fetching content of: 102_s3_token_bash.sh ---

#!/bin/bash

# 1. Check if PCAI_SECRET_TOKEN environment variable exists
if [[ -z "$PCAI_SECRET_TOKEN" ]]; then
    echo "Error: Environment variable 'PCAI_SECRET_TOKEN' is not set."
    echo "Please run: export PCAI_SECRET_TOKEN='your_token_here'"
    exit 1
fi

# 2. Check for exactly two command-line arguments
if [[ "$#" -ne 2 ]]; then
    echo "Usage: $0 <path_to_refresh_token_file> <keycloak_domain>"
    echo "Example: $0 ./tokens.txt sso.appliance.local"
    exit 1
fi

# Assign arguments to descriptive variables
export TOKEN_FILE="$1"
export KC_ADDR="$2"
export SECRET="$PCAI_SECRET_TOKEN"

# --- DEBUG START (Remove these lines once fixed) ---
 echo "DEBUG: Filename detected: $TOKEN_FILE"
 echo "DEBUG: Domain detected: $KC_ADDR"
 echo "DEBUG: SECRET detected: $SECRET"

# --- DEBUG END ---


# 3. Read the file and load it into an environment variable
if [[ -f "$TOKEN_FILE" ]]; then
    export PCAI_REFRESH_TOKEN=$(cat "$TOKEN_FILE")
